In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from tqdm import tqdm
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc2"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc2


In [4]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    # 0,  # Cell
    1  # Drug
]
device=torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )

In [5]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/240 [00:00<?, ?it/s]

epoch:   0 loss:0.489531 auc:0.5615
epoch:  20 loss:0.221061 auc:0.5444
epoch:  40 loss:0.178891 auc:0.6734
epoch:  60 loss:0.166933 auc:0.6771
epoch:  80 loss:0.154018 auc:0.7095
epoch: 100 loss:0.150552 auc:0.7137
epoch: 120 loss:0.145129 auc:0.7200
epoch: 140 loss:0.142264 auc:0.7107
epoch: 160 loss:0.142207 auc:0.7092
epoch: 180 loss:0.137917 auc:0.6492


  2%|▏         | 4/240 [00:03<03:12,  1.23it/s]

epoch: 200 loss:0.135118 auc:0.6592
Fit finished.
epoch:   0 loss:0.568385 auc:0.4004
epoch:  20 loss:0.238864 auc:0.5289
epoch:  40 loss:0.184063 auc:0.8362
epoch:  60 loss:0.171039 auc:0.8609
epoch:  80 loss:0.161997 auc:0.8784
epoch: 100 loss:0.151418 auc:0.9010
epoch: 120 loss:0.147054 auc:0.9135
epoch: 140 loss:0.144381 auc:0.9141
epoch: 160 loss:0.142638 auc:0.9146
epoch: 180 loss:0.140472 auc:0.8977
epoch: 200 loss:0.136910 auc:0.8920
epoch: 220 loss:0.135262 auc:0.9045
epoch: 240 loss:0.133145 auc:0.8963


  5%|▍         | 11/240 [00:07<02:23,  1.60it/s]

...

epoch: 240 loss:0.140133 auc:0.7269
Fit finished.
epoch:   0 loss:1.129012 auc:0.5956
epoch:  20 loss:0.289376 auc:0.8082
epoch:  40 loss:0.228878 auc:0.8372
epoch:  60 loss:0.187400 auc:0.8584
epoch:  80 loss:0.174804 auc:0.8790
epoch: 100 loss:0.167553 auc:0.8846
epoch: 120 loss:0.159534 auc:0.8960
epoch: 140 loss:0.151186 auc:0.9012
epoch: 160 loss:0.147975 auc:0.9021
epoch: 180 loss:0.145843 auc:0.9095
epoch: 200 loss:0.144965 auc:0.9144
epoch: 220 loss:0.143140 auc:0.9194
epoch: 240 loss:0.142022 auc:0.9181
epoch: 260 loss:0.141784 auc:0.9157
epoch: 280 loss:0.139707 auc:0.9209
epoch: 300 loss:0.137729 auc:0.9126
epoch: 320 loss:0.137170 auc:0.9160
epoch: 340 loss:0.135328 auc:0.9191
epoch: 360 loss:0.134386 auc:0.9246
epoch: 380 loss:0.133494 auc:0.9283
epoch: 400 loss:0.131985 auc:0.9295


 98%|█████████▊| 235/240 [05:22<00:04,  1.01it/s]

epoch: 420 loss:0.142451 auc:0.8873
Fit finished.
epoch:   0 loss:0.847488 auc:0.3860
epoch:  20 loss:0.280315 auc:0.4933
epoch:  40 loss:0.209428 auc:0.2018
epoch:  60 loss:0.182682 auc:0.3268
epoch:  80 loss:0.172375 auc:0.8519
epoch: 100 loss:0.164726 auc:0.8889
epoch: 120 loss:0.154765 auc:0.9206
epoch: 140 loss:0.147883 auc:0.9186
epoch: 160 loss:0.145591 auc:0.9098
epoch: 180 loss:0.143883 auc:0.9000
epoch: 200 loss:0.143771 auc:0.8020


100%|██████████| 240/240 [05:24<00:00,  1.35s/it]

epoch: 220 loss:0.141355 auc:0.8493
epoch: 240 loss:0.140108 auc:0.7382
Fit finished.


In [7]:
true_data_s.to_csv(f"new_drug_true_{args.data}.csv")
predict_data_s.to_csv(f"new_drug_pred_{args.data}.csv")